# Scraping Laptops data from BestBuy Canada website

**Updated for Selenium-based scraping with modular architecture**
This is an example of data scraping from BestBuy Canada website. This notebook demonstrates how to use the refactored webscraping project with Selenium WebDriver to handle JavaScript-rendered content.

This exercise helps in deciding what computer to buy for data science projects by analyzing laptop data from BestBuy. We use Selenium WebDriver to interact with the website and automatically collect data about laptops from the dynamic React-based website.

After getting the CSV file, we clean it, examine descriptive statistics, and create visualizations.

## New Project Structure

The project has been reorganized into:
- **src/** - All Python modules (config, scraper_selenium, data_cleaner, visualizer, logger)
- **data/** - CSV data files
- **notebooks/** - This tutorial notebook
- **docs/** - Documentation

## Setup: Import Required Libraries and Modules

In [ ]:
!pip install seaborn

In [ ]:
# Standard library imports
import sys
from pathlib import Path

# Add parent directory to path to import from src/
sys.path.insert(0, str(Path.cwd().parent))

# External library imports
import pandas as pd
import matplotlib.pyplot as plt

# Project module imports
from src.logger import logger
from src.config import get_config, build_url
from src.scraper_selenium import BestBuySeleniumScraper, scrape_all_laptops
from src.data_cleaner import save_data, load_and_process_data, clean_price, clean_votes
from src.visualizer import visualize_data, create_histograms, create_boxplots

%matplotlib inline

## Using the Logger

The project now uses **Loguru** for professional logging instead of print statements.

In [ ]:
# Demonstrate logging
logger.info("Starting laptop data analysis tutorial with Selenium")
logger.debug("This is a debug message")
logger.warning("This is a warning message")
logger.success("Logger is working correctly!")

## Configuration

Load the scraping configuration from the config module.

In [ ]:
# Get configuration
config = get_config()
logger.info(f"Configuration loaded: {len(config['pages'])} pages, {len(config['ram_sizes'])} RAM sizes")
print(f"Pages to scrape: {config['pages'][:5]}...")  # Show first 5
print(f"RAM sizes: {config['ram_sizes']}")
print(f"Output file: {config['output_file']}")

## Why Selenium?

Best Buy Canada's website is built with React and loads content dynamically using JavaScript. Traditional HTTP requests (like `requests` library) only get the initial HTML, which doesn't contain the product data. We need Selenium WebDriver to:

1. Open a real browser (Chrome)
2. Wait for JavaScript to execute
3. Let React render the product components
4. Extract the fully-rendered HTML

In [ ]:
# Build a test URL
test_url = build_url('1', '8')
logger.info(f"Test URL: {test_url}")

# Create a scraper instance (use headless=False to see the browser in action)
scraper = BestBuySeleniumScraper(config, headless=True)

try:
    # Setup the driver
    if scraper.setup_driver():
        logger.success("WebDriver initialized successfully!")
        
        # Load the page
        scraper.driver.get(test_url)
        logger.info("Page loaded, waiting for products...")
        
        # Wait for products to load
        if scraper.wait_for_products(timeout=15):
            logger.success("Products detected on page!")
            
            # Get containers
            from bs4 import BeautifulSoup
            page_source = scraper.driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            containers = soup.find_all('div', {'itemtype': lambda x: x and 'Product' in x})
            
            logger.info(f"Found {len(containers)} product containers")
            
            # Try to extract first product
            if containers:
                first_product = scraper.extract_laptop_data(containers[0])
                if first_product:
                    logger.success("Successfully extracted first product:")
                    for key, value in first_product.items():
                        print(f"  {key}: {value}")
                else:
                    logger.warning("Could not extract data from first container")
        else:
            logger.error("Products did not load in time")
            
finally:
    # Always close the driver
    scraper.close_driver()

## Extract Data from First Laptop

Let's examine the first laptop container to understand the data structure.

## Option 1: Run Full Scraper

**Warning:** This will make many requests and may take a long time. Consider using existing data instead.

In [ ]:
# UNCOMMENT TO RUN FULL SCRAPING (can take 20-30 minutes)
# logger.info("Starting full Selenium web scraping...")
# data = scrape_all_laptops(config, build_url)
# logger.success(f"Scraping complete! Collected {len(data['names'])} laptops")
#
# # Save the data
# output_path = config['output_file']
# df = save_data(data, output_path)
# logger.success(f"Data saved to {output_path}")

## Option 2: Load Existing Data

Let's use the existing scraped data from the data/ directory.

In [ ]:
# Load existing data
# Try the 2025 data first, fall back to 2019 data if not available
import os
data_files = [
    '../data/laptops_bestbuy_2025.csv',
    '../data/laptops_rating2019.csv',
    '../data/laptops_rating.csv'
]

df_raw = None
data_file = None

for file in data_files:
    if os.path.exists(file):
        data_file = file
        logger.info(f"Loading data from {data_file}")
        try:
            df_raw = pd.read_csv(data_file)
            logger.success(f"Data loaded successfully! Shape: {df_raw.shape}")
            break
        except Exception as e:
            logger.error(f"Error loading {file}: {e}")
            continue

if df_raw is None:
    logger.error("No data file found! Please run the scraper first or check data directory.")
else:
    display(df_raw.head())

## Data Cleaning

Use the data_cleaner module to process the raw data.

In [ ]:
# Example: Clean a single price
sample_price = '$1,234.56'
cleaned_price = clean_price(sample_price)
logger.info(f"Price cleaning example: '{sample_price}' -> {cleaned_price}")

# Example: Clean a vote count
sample_vote = '(123)'
cleaned_vote = clean_votes(sample_vote)
logger.info(f"Vote cleaning example: '{sample_vote}' -> {cleaned_vote}")

In [ ]:
# Clean the entire dataframe
logger.info("Cleaning dataframe...")
df_clean = load_and_process_data(data_file)
logger.success("Data cleaning complete!")

# Display cleaned data info
print("\nCleaned Data Info:")
df_clean.info()

In [ ]:
# Display first few rows
df_clean.head()

## Descriptive Statistics

In [ ]:
logger.info("Computing descriptive statistics...")
stats = df_clean[['prices', 'ratings', 'votes']].describe()
print("\nDescriptive Statistics:")
print(stats)

## Find Interesting Insights

In [ ]:
# Find laptop with most votes
max_votes_idx = df_clean['votes'].idxmax()
most_voted = df_clean.loc[max_votes_idx]
logger.info(f"Most voted laptop: {most_voted['laptops']} with {most_voted['votes']} votes")
print(f"\nMost Voted Laptop:")
print(f"  Name: {most_voted['laptops']}")
print(f"  Price: ${most_voted['prices']:.2f}")
print(f"  Rating: {most_voted['ratings']}%")
print(f"  Votes: {most_voted['votes']:.0f}")

In [ ]:
# Find laptops with perfect rating (100%)
perfect_ratings = df_clean[df_clean['ratings'] == 100.0]
logger.info(f"Found {len(perfect_ratings)} laptops with 100% rating")
if len(perfect_ratings) > 0:
    print("\nLaptops with 100% rating:")
    print(perfect_ratings[['laptops', 'prices', 'votes']].head())

## Data Visualization

Use the visualizer module to create charts.

In [ ]:
# Create histograms
logger.info("Creating histograms...")
create_histograms(df_clean)

In [ ]:
# Create boxplots
logger.info("Creating boxplots...")
create_boxplots(df_clean)

## Complete Visualization Pipeline

In [ ]:
# Use the complete visualization function
logger.info("Running complete visualization pipeline...")
visualize_data(df_clean)

# Remove outliers

In [ ]:

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a specified column of a DataFrame using the 1.5 * IQR rule.
    Returns a new DataFrame with outliers filtered out.
    """
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Define bounds for outlier detection (1.5 * IQR)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter the DataFrame: keep rows where the column value is within the bounds
    mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)
    return df[mask]



In [ ]:
# --- Application Logic ---

# Assuming your DataFrame is named 'df'
df_cleaned = df_clean.copy() 

# List of columns to check for outliers
columns_to_check = ['prices', 'ratings', 'votes']

for col in columns_to_check:
    # Print the bounds for logging/checking (optional)
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print(f"Processing column: {col}")
    print(f"  Bounds: ({lower_bound:.2f}, {upper_bound:.2f})")
    
# Apply the filtering function and update the DataFrame
df_cleaned = remove_outliers_iqr(df_cleaned, col)


In [ ]:
# Use the complete visualization function
logger.info("Running complete visualization pipeline...")
visualize_data(df_cleaned)

## Custom Analysis

Perform additional custom analysis on the data.

In [ ]:
# Price distribution by rating category
logger.info("Analyzing price by rating category...")

# Categorize ratings
df_clean['rating_category'] = pd.cut(
    df_clean['ratings'], 
    bins=[0, 70, 85, 95, 100],
    labels=['Low (0-70)', 'Medium (70-85)', 'High (85-95)', 'Excellent (95-100)']
)

# Group by category and show mean price
price_by_rating = df_clean.groupby('rating_category')['prices'].agg(['mean', 'median', 'count'])
print("\nPrice Statistics by Rating Category:")
print(price_by_rating)

In [ ]:
# Visualize price vs rating category
plt.figure(figsize=(12, 6))
df_clean.boxplot(column='prices', by='rating_category', figsize=(12, 6))
plt.title('Price Distribution by Rating Category')
plt.suptitle('')  # Remove automatic title
plt.ylabel('Price ($)')
plt.xlabel('Rating Category')
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Modular Code Structure**: Using functions from `src/` modules
2. **Loguru Logging**: Professional logging instead of print statements
3. **Data Pipeline**: Configuration → Scraping → Cleaning → Visualization
4. **Data Analysis**: Descriptive statistics and custom insights
5. **Visualization**: Histograms, boxplots, and custom charts

### Next Steps:

- Explore different RAM sizes and price ranges
- Add more sophisticated analysis (correlation, regression)
- Create a dashboard for interactive exploration
- Schedule regular scraping to track price trends over time

In [ ]:
logger.success("Tutorial complete! You've learned how to use Selenium-based webscraping with the updated modules.")